In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

<h4>Leemos el archivo y seleccionamos las columnas</h4>

In [14]:
df_steam_games = pd.read_csv('df_steam_games_final.csv')
df_steam_games.head(5)
columns = ['id','title','genres']

In [20]:
df_steam = df_steam_games.dropna(subset=columns)[columns]
df_steam['genres'] = df_steam['genres'].apply(eval)
df_steam['id'] = df_steam['id'].astype(int)

<h4>Elegimos una muestra del dataframe</h4>

In [21]:
tamano_muestra = 3000
# Realiza el muestreo aleatorio
df_steam_muestra= df_steam.sample(n=tamano_muestra, random_state=42)
df_steam_muestra = df_steam_muestra.reset_index(drop=True)


In [22]:
df_steam_muestra

,id,title,genres
0,707850,Jigsaw Puzzles,"[Adventure, Casual, Indie, Strategy]"
1,331520,Oblivious Garden ~Carmina Burana Soundtrack,"[Casual, Indie]"
2,567860,Passage 4,"[Casual, Indie]"
3,343599,Call of Duty®: Advanced Warfare - Jackpot Pers...,[Action]
4,322540,Galactic Inheritors,"[Indie, Strategy]"
...,...,...,...
2995,370810,Universal Combat - The Lyrius Conflict,"[Action, Indie, Simulation, Strategy]"
2996,652990,Eternal Maze,"[Adventure, Casual, Indie, RPG, Strategy]"
2997,572020,Darwin's Demons,"[Action, Casual, Indie, Simulation]"
2998,358140,Fun Theme Set,[Action]


<h4>Matriz TF-IDF</h4>

In [6]:
tfidf_vectorizer = TfidfVectorizer()
genero_matrix = tfidf_vectorizer.fit_transform(df_steam_muestra['genres'].apply(lambda x: ' '.join(x)))

In [7]:
print(genero_matrix.shape)

(3000, 32)


In [8]:
print(genero_matrix)

  (0, 26)	0.5705611690054956
  (0, 13)	0.37008522902180746
  (0, 6)	0.5178866406550827
  (0, 2)	0.5189318867770143
  (1, 13)	0.5814110733891201
  (1, 6)	0.8136099579900132
  (2, 13)	0.5814110733891201
  (2, 6)	0.8136099579900132
  (3, 1)	1.0
  (4, 26)	0.8389669947798211
  (4, 13)	0.5441823055466942
  (5, 26)	1.0
  (6, 1)	0.7765295747563586
  (6, 13)	0.6300808039677998
  (7, 1)	0.7765295747563586
  (7, 13)	0.6300808039677998
  (8, 22)	1.0
  (9, 26)	1.0
  (10, 1)	0.5824550014434435
  (10, 13)	0.4726075187795617
  (10, 6)	0.6613533885046212
  (11, 13)	0.5806350629673359
  (11, 2)	0.8141639415086607
  (12, 23)	0.6525850281006058
  (12, 26)	0.6356982971281704
  :	:
  (2990, 1)	1.0
  (2991, 21)	1.0
  (2992, 22)	0.6985024537791091
  (2992, 13)	0.41550689832394844
  (2992, 2)	0.582621952478428
  (2993, 13)	0.5814110733891201
  (2993, 6)	0.8136099579900132
  (2994, 22)	0.674982593462257
  (2994, 26)	0.6190180819497341
  (2994, 13)	0.4015160180394273
  (2995, 23)	0.5817752509216527
  (2995, 1)	0

In [9]:
cosine_sim = cosine_similarity(genero_matrix, genero_matrix)

In [10]:
print(cosine_sim)

[[1.         0.63652938 0.63652938 ... 0.41433128 0.         0.51893189]
 [0.63652938 1.         1.         ... 0.65092247 0.         0.        ]
 [0.63652938 1.         1.         ... 0.65092247 0.         0.        ]
 ...
 [0.41433128 0.65092247 0.65092247 ... 1.         0.46641694 0.        ]
 [0.         0.         0.         ... 0.46641694 1.         0.        ]
 [0.51893189 0.         0.         ... 0.         0.         1.        ]]


In [11]:
def get_recommendations(id, cosine_sim=cosine_sim):
    idx = df_steam_muestra[df_steam_muestra['id'] == id].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    top_indices = [i for i, _ in sim_scores[1:6]]
    
    return df_steam_muestra['title'].iloc[top_indices]


In [23]:
dicc = {'Juegos:' : get_recommendations(370810).values.tolist()}

In [24]:
dicc

{'Juegos:': ['Zulu Response',
  'Universal Combat - The Lyrius Conflict',
  'Arma 2: Private Military Company',
  'Arma 3 DLC Bundle 1',
  'Arma 3 Malden']}

In [25]:
df_steam_muestra[df_steam_muestra['title']=='Zulu Response']

,id,title,genres
2544,401250,Zulu Response,"[Action, Indie, Simulation, Strategy]"


In [ ]:
df_steam_muestra.iloc[75]

title                                       The Vagrant
genres    [Action, Adventure, Indie, RPG, Early Access]
Name: 75, dtype: object